[Home](../index.ipynb) / Web-Server: Joystick Sample
***

# Beispiel: "Joystick"
Joystick-HTML: Idea from https://www.kirupa.com/html5/drag.htm

Nach dem Start der Programme das WLAN des Controllers auswählen und die auf der Console angezeigte IP-Adresse ansurfen (meist 192.168.4.1).

**Relevanter Code:**  

```python
class HttpServer( SimpleAccessPointHttpServer ):
    [some code]
    
    def dispatch( self, strPath ) :
       [some code]
            # Here You have direction in[-1;1] and speed in [-1;1].
            # Change this for Your needs.
            print( "Set: direction = {}, speed = {}".format( iDirection, iSpeed ) )
            
            return True
       [some code]
```

Statt dem `print` kann eigener sinnvoller Code verwendet werden.  
Für den echten Betrieb die `print` Anweisungen entfernen, die verlangsamen nur das Programm.

In [ ]:
#%serialconnect --port=COM3 # Windows
%serialconnect # Linux or Windows with one COM port 


#================================================

MAX_REQUEST_LENGTH = 1024

#================================================

import network
import socket
import info
import ubinascii
import gc

#================================================
  
class SimpleAccessPointHttpServer:
    def __init__( self ) :
        self.objSocket  = None
        self.ipAddress  = "?.?.?.?"
        self.subnetMask = self.ipAddress
        self.gateway    = self.ipAddress
        self.dnsServer  = self.ipAddress
        self.macAddress = "?:?:?:?:?:?"

    def __str__( self ) :
        return "Name: " + self.__class__.__name__ + ", IP address: " + self.ipAddress + ", Subnet mask: " + self.subnetMask + ", Gateway: " + self.gateway + ", DNS server: " + self.dnsServer + ", MAC address: " + self.macAddress
        
    def start( self, ssid = "ESP32_Ajax_Sample", password = "1231231123", maxClients = 5 ):
        self.wlanAccessPoint = network.WLAN( network.AP_IF ) # create access-point interface

        if info.TYPE == "Esp8266 Croduino Nova" :
            self.wlanAccessPoint.config( essid = ssid, password = password ) # Esp8266
        else :
            self.wlanAccessPoint.config( essid = ssid, password = password, max_clients = maxClients ) # ESP32

        self.wlanAccessPoint.active( True )                  # activate the interface

        print( self.wlanAccessPoint.ifconfig()[0] )

        self.objSocket = socket.socket( socket.AF_INET, socket.SOCK_STREAM )

        try:
            self.objSocket.bind( ( '', 80 ) )
            self.objSocket.listen( maxClients )
            
            self.ipAddress, self.subnetMask, self.gateway, self.dnsServer = self.wlanAccessPoint.ifconfig() # [IP address, subnet mask, gateway, DNS server]
            self.macAddress = ubinascii.hexlify( network.WLAN().config('mac'),':' ).decode()
            
        except Exception as e:
            #print( "Exception of type {0} occurred. Arguments:\n{1!r}".format(type(e).__name__, e.args))
            print( e )

        gc.collect()

        
    def stop( self ):
        try : self.objSocket.close()
        except: pass
    
        try : self.wlanAccessPoint.active( False ) 
        except: pass
    
        gc.collect()

        
        
    def accept( self ):
        try :
            # socket.accept(): The return value is a pair (socketFrom, addressFrom)
            # where socketFrom is a new socket object usable to send 

            self.socketFrom = self.objSocket.accept()[ 0 ]
            
            if not self.dispatch(
                    self.getRequestPath( 
                        str( self.socketFrom.recv( MAX_REQUEST_LENGTH ) )
                    )
                ) :
                self.socketFrom.send( "HTTP/1.1 404 Not Found\n" )
                self.socketFrom.send( "Connection: close\n\n"    )

            
        except Exception as _e:
            print( _e )
            #print( "Exception of type {0} occurred. Arguments:\n{1!r}".format(type(_e).__name__, _e.args))
        
        finally :
            try : self.socketFrom.close()
            except: pass

    def sendHttpHeader( self, strContentType ) :
        self.send( "HTTP/1.1 200 OK\n"     )
        self.send( "Content-type: text/"   )
        self.send( strContentType          )
        self.send( "; charset=UTF-8\n"     )
        self.send( "Connection: close\n\n" )

    def sendHtmlHttpHeader( self ) :
        self.sendHttpHeader( "html" )
        
    def sendAjaxHttpHeader( self ) :
        self.sendHttpHeader( "plain" )
                    
        
    def getRequestPath( self, strRequest ) :
        iRequestPathStart = strRequest.find( "GET " )

        if iRequestPathStart >= 0 :
            iRequestPathStart += 4
            
            iRequestPathEnd = strRequest.find( " ", iRequestPathStart )

            if iRequestPathEnd >= 0 :
                return strRequest[ iRequestPathStart : iRequestPathEnd ]
            
        return "/"
    
    
    def dispatch( self, strPath ) :
        return True
        
   
    def sendHtmlHead( self, strTitle = "AP", strJavaScript="", strCss = "", strAdditionalTags = "" ) :
        self.send( '<!DOCTYPE html><html lang="de"><head><meta charset="utf-8"><title>' )
        self.send( strTitle )
        self.send( '</title><meta name="viewport" content="width=device-width, initial-scale=1"><link rel="icon" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAIAAACQd1PeAAAADElEQVQI12P4//8/AAX+Av7czFnnAAAAAElFTkSuQmCC">' )
        self.send( '<script type="text/javascript">' )
        self.send( strJavaScript )
        self.send( '</script>' )
        self.send( '<style>' )
        self.send( strCss )
        self.send( '</style>' )
        self.send(  strAdditionalTags  )
        self.send( '</head>' )

    def send( self, strText ) :
        self.socketFrom.write( strText )


#==========================================================



import re
# CSS

STR_CSS = r'''
html, body {margin: 0;padding: 0;width: 100vw;height: 100vh;display: flex;flex-direction: row;align-items: center;justify-content: center;text-align: center;background-color: #AAAAAA;}
    #idMainContainer {width: 80vh;height: 80vw;max-width: 80vw;max-height: 80vh;background-color: #AAAAAA;}
    #idStickContainer {width: 100%;height: 100%;background-color: #EEEEAA;display: flex;align-items: center;justify-content: center;overflow: hidden;touch-action: none;border-radius: 50%;border: 3px solid black;}
    #idStick {width: 80px;height: 80px;background-color: #AAAAFF;border: 10px solid #4444FF;border-radius: 50%;touch-action: none;user-select: none;}
    #idStick:active {background-color: #EEEEFF;}
    #idStick:hover {cursor: pointer;border: 12px solid #4444AA;}
    .button{background-color:#4CAF50;border:none;color:white;padding:16px 40px;text-decoration:none;font-size:30px;margin:2px;cursor:pointer;}
'''


# JavaScript: Sends Ajax-requests.

STR_JAVASCRIPT = r'''
document.addEventListener(
    'DOMContentLoaded',
    function() { new JoystickApp(); },
    false
);


class JoystickApp
{
    constructor()
    {
        let objThis = this;

        this.isActive = false;
        this.posX   = 0;
        this.posY   = 0;
        this.startX = 0;
        this.startY = 0;

        this.htmlMainContainer  = document;
        this.htmlStickContainer = document.getElementById( "idStickContainer" );
        this.htmlStick          = document.getElementById( "idStick");

        this.radiusMax = this.htmlStickContainer.clientWidth/2 - 40;

        this.htmlMainContainer.addEventListener("touchstart", this.dragStart.bind(this), false);
        this.htmlMainContainer.addEventListener("touchend",   this.dragEnd.bind(this), false);
        this.htmlMainContainer.addEventListener("touchmove",  this.drag.bind(this), false);

        this.htmlMainContainer.addEventListener("mousedown",  this.dragStart.bind(this), false);
        this.htmlMainContainer.addEventListener("mouseup",    this.dragEnd.bind(this), false);
        this.htmlMainContainer.addEventListener("mousemove",  this.drag.bind(this), false);

        this.isWaiting = false; // only to avoid "overload"
        this.posXLast = 0;
        this.posYLast = 0;
        

        this.htmlButtonStop = document.getElementById( "idButtonStop" );

        this.htmlButtonStop.addEventListener(
            "click",
            function() {
                objThis.setTranslate(objThis.startX,objThis.startY);
            }
        )
        
    }

    dragStart( e )
    {
        if (e.target === this.htmlStick)
        {
            let ePointer = (e.type === "touchstart") ? e.touches[0] : e;

            this.startX = ePointer.clientX - this.posX;
            this.startY = ePointer.clientY - this.posY;

            this.isActive = true;
        }
    }

    drag(e)
    {
        if (this.isActive)
        {
            e.preventDefault();

            let ePointer = (e.type === "touchmove") ? e.touches[0] : e;

            this.setTranslate( ePointer.clientX, ePointer.clientY );
        }
    }

    dragEnd(e)
    {
        this.initialX = this.currentX;
        this.initialY = this.currentY;

        this.isActive = false;
    }

    setTranslate( clientX, clientY )
    {
        this.posX = clientX - this.startX;
        this.posY = clientY - this.startY;

        let radiusPos = Math.sqrt( this.posX**2 + this.posY**2 );

        if ( radiusPos > this.radiusMax )
        {
            this.posX *= this.radiusMax/radiusPos
            this.posY *= this.radiusMax/radiusPos
        }


        this.htmlStick.style.transform = "translate3d(" + this.posX + "px, " + this.posY + "px, 0)";

        let objThis = this;
        let hTimeOut = null
        // Ajax:
        let xhttp = new XMLHttpRequest();
        xhttp.onreadystatechange = function() {
            if (this.readyState == 4 && this.status == 200) {
                objThis.isWaiting = false;
                try {clearTimeout( hTimeOut )} catch (_e) {}
                document.getElementById("idOutput").innerHTML = this.responseText;
            }
        };

        if ( ! this.isWaiting )
        {
            if ( this.posX != this.posXLast || this.posY != this.posYLast )
            {
                this.posXLast = this.posX
                this.posYLast = this.posY
                objThis.isWaiting = true;
                hTimeOut = setTimeout(function(){objThis.isWaiting=false;},2000)
                
                xhttp.open("GET", "/joystick/" + (this.posX/this.radiusMax) + "_" + (-this.posY/this.radiusMax), true);
                xhttp.send();
            }
        }

    }
}
'''


STR_BODY = r'''
<body>
  <div id="idMainContainer">
    <div id="idStickContainer">
      <div id="idStick"></div>
    </div>
    <div id="idBottom">
      <p><input type="button" class="button" id="idButtonStop" value="Stop"/></p>
      <p id="idOutput"></p>
    </div>
  </div>
</body>
'''

PATTERN_JOYSTICK = re.compile( "/joystick/(.*)_(.*)" )

#######################################
# You have to owerwrite dispatch()

class HttpServer( SimpleAccessPointHttpServer ):
    def __init__( self ) :
        super().__init__()

    def dispatch( self, strPath ) :
        print( "dispatch: ", strPath )

        match = PATTERN_JOYSTICK.match( strPath )
        
        
        if match :
            iDirection = float(match.group( 1 ))
            iSpeed     = float(match.group( 2 ))
            
            self.sendAjaxHttpHeader()
            self.send( match.group( 1 ) + ";" + match.group( 2 ) )
            
            # Here You have direction in[-1;1] and speed in [-1;1].
            # Change this for Your needs.
            print( "Set: direction = {}, speed = {}".format( iDirection, iSpeed ) )
            
            return True
        

        # All other requests are maped to "/"
        self.sendHtmlHttpHeader() 
        self.sendHtmlHead( strTitle = "Joystick Remote Control", strJavaScript = STR_JAVASCRIPT, strCss = STR_CSS )
        self.send( STR_BODY )
        self.send( "</html>" )
            
        return True

    
httpServer = HttpServer()
httpServer.start( ssid = "ESP32_Joystick_Sample", password = "1231231123" )
print( httpServer )


#######################################
# Loop


try :
    while True:
        print( "Waiting for client...", end="" )
        httpServer.accept()
        
except KeyboardInterrupt:
    httpServer.stop()
    print( "Server stopped." )